In [8]:
from __future__ import annotations
import typing
from dataclasses import dataclass
import warnings
from contextlib import contextmanager

In [9]:
import numpy as np

In [10]:
@contextmanager
def localize_globals(*exceptions: str, restore_values: bool = True):
    exceptions: typing.Set[str] = set(exceptions)

    old_globals: typing.Dict[str, typing.Any] = dict(globals())
    allowed: typing.Set[str] = set(old_globals.keys())
    allowed.update(exceptions)

    yield None

    new_globals: typing.Dict[str, typing.Any] = globals()

    for name in tuple(new_globals.keys()):
        if name not in allowed:
            del new_globals[name]
    
    if not restore_values:
        return
    
    new_globals.update(
        {k: v for k, v in old_globals.items() if k not in exceptions}
    )

In [11]:
@dataclass(frozen=True)
class Equation:
    A: np.ndarray
    b: np.ndarray
    
    @staticmethod
    def random(size: int) -> Equation:
        A: np.ndarray = np.random.rand(size, size)
        b: np.ndarray = np.random.rand(size)
        
        return Equation(A, b)
    
    @staticmethod
    def random_many(size: int, *, count: int) -> typing.Iterable[Equation]:
        return (Equation.random(size) for _ in range(count))

In [12]:
@dataclass(frozen=True)
class Solution:
    L: np.ndarray
    U: np.ndarray
    Q: np.ndarray
    x: np.ndarray

In [13]:
def matrix_swap(arr: np.ndarray, i: int, j: int,
                axis: int = 0) -> None:
    if i == j:
        return
    
    # Doesn't affect the contents
    arr = arr.swapaxes(0, axis)
    
    arr[[i, j]] = arr[[j, i]]

In [14]:
def solve_gauss(equation: Equation) -> Solution:
    A: np.ndarray = equation.A
    b: np.ndarray = equation.b
    
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    
    size: int = A.shape[0]
    
    L: np.ndarray = np.eye(size)
    U: np.ndarray = A.copy()
    Q: np.ndarray = np.eye(size)
    
    for row_idx in range(size):
        pivot: int = np.argmax(np.abs(U[row_idx:, row_idx])) + row_idx
        
        matrix_swap(U, row_idx, pivot, axis=1)
        matrix_swap(Q, row_idx, pivot, axis=1)
        
        normalization_coeff: float = U[row_idx, row_idx]
        if np.abs(normalization_coeff) < 1e-10:
            warnings.warn("Matrix is close to singular")
        
        A[row_idx] /= normalization_coeff
        L_delta_row: np.ndarray = L[row_idx] / normalization_coeff
        
        for other_row_idx in range(row_idx + 1, size):
            L[other_row_idx] -= L_delta_row * U[other_row_idx, row_idx]
            U[other_row_idx] -= U[row_idx] * U[other_row_idx, row_idx]
    
    return Solution(L, U, Q, ...)

In [19]:
equation: Equation = Equation.random(3)

In [22]:
true_solution: np.ndarray = np.linalg.solve(equation.A, equation.b)

In [23]:
equation.A @ true_solution - equation.b

array([ 1.11022302e-16, -4.16333634e-17, -1.11022302e-16])

In [17]:
def compare_on(equation: Equation) -> float:
    my_result: np.ndarray = solve_gauss(equation).x
    np_result: np.ndarray = np.linalg.solve(equation.A, equation.b)
    
    return np.linalg.norm(my_result - np_result)

In [18]:
with localize_globals():
    for equation in Equation.random_many(3, count=10):
        print(compare_on(equation))

TypeError: unsupported operand type(s) for -: 'ellipsis' and 'float'